# Chinook Media Shop

In the following is a example implementation of the Chinook Media Shop. Here we would describe the applications domain and project scope.

# Imports

Import standard libs included in python

In [1]:
import shutil
import os

Import third party libs

In [2]:
import pandas as pd

Import own modules

In [3]:
import model
import data_access
import business_logic
import ui.input_helper as input_helper

# Using the domain models

In the following we are using the domain models without any managers or persistence.

In [4]:
artist: model.Artist = model.Artist(1, "New Artist")
album: model.Album = model.Album(1, "New Album", artist)
track: model.Track = model.Track(1, "New Track", model.MediaType.MPEG, 10000, 1.99, album=album)
print(artist)
for album in artist.albums:
    print(album)
    for track in album.tracks:
        print(track)

print(20*"*")

track.album = None
album.artist = None
print(artist)
print(album)
print(track)
print()
print(20*"*")
print(artist)
for album in artist.albums:
    print(album)
    for track in album.tracks:
        print(track)

artist.add_album(album)
album.add_track(track)
print()
print(20*"*")
print(artist)
for album in artist.albums:
    print(album)
    for track in album.tracks:
        print(track)

# Configure the application

Application Configuration
The following section sets up the database configuration.

First, the source database is duplicated to create a working database, ensuring a fresh start whenever the notebook is executed. Next, environment variables, such as `DB_FILE`, are defined to allow other parts of the application, like the `data_access` layer, to access these values dynamically.

Using environment variables offers flexibility, as they can be configured within a file or directly at the system level. In this case, they are set programmatically using `os.environ["DB_FILE"] = db_file`.

In [5]:
source = "database/media_store_o.db"
db_file = "database/working_db.db"
shutil.copyfile(source, db_file) #Copy original DB first to always start fresh.

### Configure Environment Variables

In [6]:
os.environ["DB_FILE"] = db_file

### Configure display for pandas

In [7]:
# Increase row and column display limits
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)  # Auto-adjust display width
pd.set_option("display.max_colwidth", None)  # Show full column content

## Manager initialization

In this section, all managers from the business logic are initialized. These managers handle the application's core functionality, allowing us to execute user stories while keeping the logic separate from the UI. Additionally, they manage interactions with the persistence layer.

In [8]:
artist_manager = business_logic.ArtistManager()
album_manager = business_logic.AlbumManager()
track_manager = business_logic.TrackManager()
genre_manager = business_logic.GenreManager()

# User Stories

## Get all the artists

In [9]:
artists = artist_manager.read_all_artists()
for artist in artists:
    album_manager.read_artists_albums(artist)
    print(artist)

### Get all artists as DataFrame
Now lets use a DataFrame and see how we can do the same with a table.

In [10]:
artist_df = artist_manager.read_all_artists_as_df()
artist_df

## Find artists by similar name

In [11]:
artist_name = input_helper.input_valid_string("Artist Name")

artists = artist_manager.read_artists_by_similar_name(artist_name)
artists.sort(key=lambda a: a.name)
for artist in artists:
    print(artist)

### Find artists by similar name as DataFrame
Now lets use a DataFrame and see how we can do the same with a table.

In [12]:
artist_name = input_helper.input_valid_string("Artist Name")

artists = artist_manager.read_artists_by_similar_name_as_df(artist_name).sort_values("Name")
artists

## Find artist by ID

In [13]:
artist_id = None
cancel = False
while not artist_id and not cancel:
    try:
        artist_id = input_helper.input_valid_int("Artist ID:")
    except input_helper.EmptyInputError:
        cancel = True
    except ValueError as err:
        print(err)

if artist_id is not None:
    loaded_artist = artist_manager.read_artist(artist_id)
    if loaded_artist:
        print(loaded_artist)
        album_manager.read_artists_albums(loaded_artist)
        for i_album, album in enumerate(loaded_artist.albums, start=1):
            print("\t", f"{i_album}.", album)
            track_manager.read_tracks_by_album(album)
            for i_track, track in enumerate(album.tracks, start=1):
                print("\t\t", f"{i_track}.", track)
    else:
        print("Artist not found.")
else:
    print("No Artist ID provided.")

## Create new artist

In [16]:
artist_name = input_helper.input_valid_string("Artist Name")
if artist_name:
    new_artist = artist_manager.create_artist(artist_name)
    print(new_artist)
    
    create_album = input_helper.input_y_n(f"Create new Album for artist '{new_artist}' (y/n) [y]?", default=input_helper.YesOrNo.YES)
    created_albums = 0
    while create_album:
        album_name = input_helper.input_valid_string("Album Name")
        if album_name:
            new_album = album_manager.create_album(album_name, new_artist)
            created_albums += 1
            print("Created Album!")
            print("\t", f"{created_albums}.", new_album)
            create_track = input_helper.input_y_n(f"Create new Track for Album {new_album} (y/n) [y]?", default=input_helper.YesOrNo.YES)
            created_tracks = 0
            while create_track:
                track_name = input_helper.input_valid_string("Track Name")
                if track_name:
                    media_type = None
                    while not media_type:
                        try:
                            media_type_id = input_helper.input_valid_int("Media Type ID", min_value=1, max_value=5)
                            media_type = model.MediaType.from_id(media_type_id)
                        except Exception as err:
                            print(err)
                            print("Use the ID of one of the following Media Types:")
                            for mt in model.MediaType:
                                print(mt.media_type_id, mt.name)
                    milliseconds = None
                    while not milliseconds:
                        try:
                            milliseconds = input_helper.input_valid_int("Milliseconds", min_value=10000)
                        except input_helper.OutOfRangeError as err:
                            print(err)
                            print("You can't add a track with less than 10000 milliseconds (10 sec).")
                        except Exception as err:
                            print(err)
                            
                    unit_price = None
                    while not unit_price:
                        try:
                            unit_price = input_helper.input_valid_float("Unit Price", min_value=0.1)
                        except input_helper.OutOfRangeError as err:
                            print(err)
                            print("You can't add a track which costs less than 0.1.")
                        except Exception as err:
                            print(err)

                    genre = None
                    none_gange = False
                    while not genre and not none_gange:
                        try:
                            genre_name = input_helper.input_valid_string("Genre")
                            if genre_name:
                                genre = genre_manager.read_genre_by_name(genre_name)
                                if not genre:
                                    available_genre = genre_manager.read_all_genre()
                                    print("Genre not found.")
                                    print("Use the name of one of the following genres:")
                                    for g in available_genre:
                                        print(g)
                            else:
                                none_gange = True
                        except Exception as err:
                            print(err)
                    new_track = track_manager.create_track(
                        track_name,
                        media_type,
                        milliseconds,
                        unit_price,
                        new_album
                    )
                    created_tracks += 1
                    print("Created Track!")
                    print("\t\t", f"{created_tracks}.", new_track)
                    create_track = input_helper.input_y_n(
                        f"Create another new Track for Album {new_album} (y/n) [y]?",
                        default=input_helper.YesOrNo.YES
                    )
                else:
                    print("No track name given!")
                    create_track = input_helper.input_y_n(
                        f"Create another new Track for Album {new_album} (y/n) [y]?", 
                        default=input_helper.YesOrNo.YES
                    )
            
            create_album = input_helper.input_y_n(
                f"Create another new Album for artist '{new_artist}' (y/n) [y]?", 
                default=input_helper.YesOrNo.YES
            )
        else:
            print("No album name given!")
            create_album = input_helper.input_y_n(
                f"Create new Album for artist '{new_artist}' (y/n) [y]?", 
                default=input_helper.YesOrNo.YES
            )
else:
    print("No artist name given!")
    print("Canceled creating an artist.")

## Update Artist

In [22]:
artist_id = None
cancel = False
while artist_id is None and not cancel:
    try:
        artist_id = input_helper.input_valid_int("Artist ID:")
        artist = artist_manager.read_artist(artist_id)
        if artist:
            print(artist)
            print("Editing Artist...")
            artist_name = input_helper.input_valid_string("Artist Name")
            if artist_name:
                artist.name = artist_name
                artist_manager.update_artist(artist)
                print(f"Artist with id {artist_id} updated.")
                updated_artist = artist_manager.read_artist(artist_id)
                print(updated_artist)        
        else:
            print(f"Arist with id {artist_id} not found.")
            artist_id = None
    except input_helper.EmptyInputError as e:
        cancel = True
        print("Canceled changing an artist.")
    except ValueError as e:
        print("Invalid Artist ID provided.")

## Delete Artist

In [32]:
artist_id = None
cancel = False
while artist_id is None and not cancel:
    try:
        artist_id = input_helper.input_valid_int("Artist ID:")
        artist = artist_manager.read_artist(artist_id)
        
        if artist:
            print(artist)
            yes_no = None
            cancel = False
            while not yes_no and not cancel:
                try:
                    yes_no = input_helper.input_y_n("Delete Artist (y/n) [n]?", default=input_helper.YesOrNo.NO)
                    if yes_no:
                        artist_manager.delete_artist(artist)
                        print(f"Artist with id {artist_id} deleted.")
                    else:
                        print("Canceled deleting an artist.")
                        cancel = True
                except Exception as e:
                    print(e)
                 
        else:
            print(f"Arist with id {artist_id} not found.")
            artist_id = None
    except input_helper.EmptyInputError as e:
        cancel = True
        print("Canceled changing an artist.")
    except ValueError as e:
        print("Invalid Artist ID provided.")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a2bdea36-0144-499d-8063-c73fb2a08fa1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>